# Importar Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import OperationalError
import os
from dotenv import load_dotenv

# Definindo parâmetros para a geração de dados

In [2]:
np.random.seed(0)  # Para garantir a reprodutibilidade
tipos_cerveja = ["Pilsen", "IPA", "Stout", "Lager", "Ale", "Wheat Beer", "Porter"]
quantidade_min, quantidade_max = 500, 5000  # Quantidade produzida por lote

# Gerando dados de produção (Lotes)

In [3]:
dados_producao = pd.DataFrame({
    "ID_Lote": range(1, 251),
    "Data_Producao": pd.date_range(start="2020-01-01", periods=250, freq='D'),
    "Tipo_Cerveja": np.random.choice(tipos_cerveja, 250),
    "Quantidade_Produzida": np.random.randint(quantidade_min, quantidade_max, 250),
    "Custo_Producao": np.random.uniform(1000, 5000, 250),  # Adicionando custo de produção
    "Lote_Ingredientes": np.random.randint(100, 200, 250),  # Adicionando lote de ingredientes
    "Tempo_Fermentacao": np.random.randint(1, 14, 250),  # Adicionando tempo de fermentação em dias
    "Data_Validade": pd.date_range(start="2021-01-01", periods=250, freq='30D')  # Adicionando data de validade
})

# Dados de vendas (2000 entradas ao longo de 3 anos)

In [6]:
dados_vendas = pd.DataFrame({
    "ID_Venda": range(1, 2001),
    "Data_Venda": pd.date_range(start="2020-01-01", periods=2000, freq='D'),
    "ID_Cliente": np.random.randint(1, 500, 2000),  # Adicionando ID do cliente
    "ID_Lote": np.random.choice(dados_producao["ID_Lote"], 2000),
    "Quantidade_Vendida": np.random.randint(1, 100, 2000),
    "Preco_Venda": np.random.uniform(5, 50, 2000),
    "Canal_Venda": np.random.choice(["Loja Física", "Online", "Distribuidor"], 2000)  # Adicionando canal de venda
})

# Dados de devolução (13% das vendas)

In [7]:
quantidade_devolucoes = int(0.13 * len(dados_vendas))
dados_devolucao = dados_vendas.sample(n=quantidade_devolucoes).copy()
dados_devolucao = dados_devolucao.rename(columns={"Quantidade_Vendida": "Quantidade_Devolvida"})
dados_devolucao["ID_Devolucao"] = range(1, quantidade_devolucoes + 1)
dados_devolucao["Motivo_Devolucao"] = np.random.choice(["Defeito", "Insatisfação", "Outro"], quantidade_devolucoes)
dados_devolucao["Estado_Produto"] = np.random.choice(["Bom", "Danificado", "Expirado"], quantidade_devolucoes)
dados_devolucao["Reembolso_Emitido"] = np.random.uniform(5, 50, quantidade_devolucoes)

# Dados de feedback (gerados aleatoriamente)

In [10]:
quantidade_feedbacks = random.randint(100, 300)
dados_feedback = pd.DataFrame({
    "ID_Feedback": range(1, quantidade_feedbacks + 1),
    "Data_Feedback": pd.date_range(start="2020-01-01", periods=quantidade_feedbacks, freq='D'),
    "ID_Cliente": np.random.randint(1, 500, quantidade_feedbacks),
    "Tipo_Feedback": np.random.choice(["Produto", "Serviço", "Entrega"], quantidade_feedbacks),
    "Descricao_Feedback": np.random.choice(["Excelente", "Bom", "Médio", "Ruim", "Péssimo"], quantidade_feedbacks),
    "Resposta_Cervejaria": np.random.choice(["Resolvido", "Pendente", "Ignorado"], quantidade_feedbacks)
})

## Inserindo dados faltantes no dataframe de feedback



In [29]:
# Adicionando a coluna id_venda com valores aleatórios
dados_feedback['id_venda'] = np.random.randint(1, 2001, len(dados_feedback))

# Adicionando a coluna classificacao com valores aleatórios (classificação de 1 a 5)
dados_feedback['classificacao'] = np.random.randint(1, 6, len(dados_feedback))

dados_feedback = dados_feedback[[
    'ID_Feedback', 'id_venda', 'Data_Feedback', 'classificacao', 
    'Tipo_Feedback', 'Descricao_Feedback', 'Resposta_Cervejaria'
]]


#### corrigindo feedback 
Excluindo ID_Cliente pois ocorreu conflito.

In [16]:
# Supondo que 'dados_feedback' já foi criado com a coluna 'ID_Cliente'
# Remover a coluna 'ID_Cliente'
if 'ID_Cliente' in dados_feedback.columns:
    dados_feedback = dados_feedback.drop(columns=['ID_Cliente'])

In [23]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtém as informações de conexão do arquivo .env
host = os.getenv('DB_HOST', 'localhost')
porta = os.getenv('DB_PORT', '5432')
dbname = os.getenv('DB_NAME', 'postgres')
usuario = os.getenv('DB_USER', 'postgres')
senha = os.getenv('DB_PASSWORD', 'default_password')

def create_connection(host_name, port, database_name, user_name, user_password):
    connection = None
    try:
        connection = psycopg2.connect(
            host=host_name,
            port=port,
            dbname=database_name,
            user=user_name,
            password=user_password
        )
        print("Conexão com o PostgreSQL estabelecida")
    except OperationalError as e:
        print(f"Ocorreu um erro ao conectar ao PostgreSQL: {e}")
    return connection

# Estabelecendo a conexão
conn = create_connection(host, porta, dbname, usuario, senha)

# Lembre-se de fechar a conexão quando terminar
# conn.close()

Conexão com o PostgreSQL estabelecida


In [18]:
def create_tables(connection):
    cursor = connection.cursor()

    # Criação da tabela de produção
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Producao (
        ID_Lote INTEGER PRIMARY KEY,
        Tipo_Cerveja TEXT,
        Data_Producao DATE,
        Quantidade_Produzida INTEGER,
        Custo_Producao REAL,
        Lote_Ingredientes TEXT,
        Tempo_Fermentacao INTEGER,
        Data_Validade DATE
    )
    """)

    # Criação da tabela de vendas
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Vendas (
        ID_Venda INTEGER PRIMARY KEY,
        Data_Venda DATE,
        ID_Cliente INTEGER,
        ID_Lote INTEGER,
        Quantidade_Vendida INTEGER,
        Preco_Venda REAL,
        Canal_Venda TEXT,
        FOREIGN KEY (ID_Lote) REFERENCES Producao (ID_Lote)
    )
    """)

    # Criação da tabela de devoluções
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Devolucoes (
        ID_Devolucao INTEGER PRIMARY KEY,
        ID_Venda INTEGER,
        Data_Devolucao DATE,
        Motivo_Devolucao TEXT,
        Quantidade_Devolvida INTEGER,
        Estado_Produto TEXT,
        Reembolso_Emitido REAL,
        FOREIGN KEY (ID_Venda) REFERENCES Vendas (ID_Venda)
    )
    """)

    # Criação da tabela de feedback
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Feedback (
        ID_Feedback INTEGER PRIMARY KEY,
        Data_Feedback DATE,
        ID_Cliente INTEGER,
        Tipo_Feedback TEXT,
        Descricao_Feedback TEXT,
        Resposta_Cervejaria TEXT,
        ID_Venda INTEGER
    )
    """)

    connection.commit()
    cursor.close()

In [19]:
def insert_data(df, table_name, connection):
    cursor = connection.cursor()
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    try:
        for row in df.itertuples(index=False, name=None):
            cursor.execute(query, row)
        connection.commit()
    except psycopg2.DatabaseError as e:
        print(f"Erro ao inserir dados na tabela {table_name}: {e}")
        connection.rollback()  # Reverter a transação em caso de erro
    finally:
        cursor.close()

# Inserir dados
insert_data(dados_producao, 'Producao', conn)
insert_data(dados_vendas, 'Vendas', conn)
insert_data(dados_devolucao[['ID_Devolucao', 'ID_Venda', 'Quantidade_Devolvida', 'Motivo_Devolucao']], 'Devolucoes', conn)
insert_data(dados_feedback, 'Feedback', conn)

Erro ao inserir dados na tabela Producao: duplicate key value violates unique constraint "producao_pkey"
DETAIL:  Key (id_lote)=(1) already exists.

Erro ao inserir dados na tabela Vendas: duplicate key value violates unique constraint "vendas_pkey"
DETAIL:  Key (id_venda)=(1) already exists.

Erro ao inserir dados na tabela Devolucoes: duplicate key value violates unique constraint "devolucoes_pkey"
DETAIL:  Key (id_devolucao)=(1) already exists.



In [20]:
conn.close()

In [21]:
print(dados_devolucao)

      ID_Venda Data_Venda  ID_Cliente  ID_Lote  Quantidade_Devolvida  \
867        868 2022-05-17         393      218                    47   
1652      1653 2024-07-10          15       72                    55   
1888      1889 2025-03-03         400      172                    99   
1779      1780 2024-11-14         219      171                    94   
137        138 2020-05-17         333       27                    34   
...        ...        ...         ...      ...                   ...   
1400      1401 2023-11-01          80       34                    90   
18          19 2020-01-19          54      243                    94   
1232      1233 2023-05-17          37      110                    20   
763        764 2022-02-02         449      139                    20   
1855      1856 2025-01-29         422      143                    91   

      Preco_Venda   Canal_Venda  ID_Devolucao Motivo_Devolucao Estado_Produto  \
867     22.332497   Loja Física             1         

### Atualizando dados da tabela devolução

In [24]:
def update_devolucoes(data, connection):
    cursor = connection.cursor()
    query = """
    UPDATE Devolucoes
    SET Estado_Produto = %s, Reembolso_Emitido = %s
    WHERE ID_Devolucao = %s;
    """
    try:
        for idx, row in data.iterrows():
            cursor.execute(query, (row['Estado_Produto'], row['Reembolso_Emitido'], row['ID_Devolucao']))
        connection.commit()
    except psycopg2.DatabaseError as e:
        print(f"Erro ao atualizar dados na tabela Devolucoes: {e}")
        connection.rollback()  # Reverter a transação em caso de erro
    finally:
        cursor.close()

# Chame esta função para atualizar a tabela Devolucoes
update_devolucoes(dados_devolucao, conn)


In [36]:
print(dados_feedback)

     ID_Feedback  id_venda Data_Feedback  classificacao Tipo_Feedback  \
0              1      1285    2020-01-01              5       Produto   
1              2       186    2020-01-02              4       Entrega   
2              3      1403    2020-01-03              5       Serviço   
3              4      1210    2020-01-04              5       Entrega   
4              5      1115    2020-01-05              3       Entrega   
..           ...       ...           ...            ...           ...   
138          139      1430    2020-05-18              1       Entrega   
139          140      1582    2020-05-19              2       Serviço   
140          141      1180    2020-05-20              2       Entrega   
141          142         9    2020-05-21              3       Serviço   
142          143      1342    2020-05-22              4       Entrega   

    Descricao_Feedback Resposta_Cervejaria  
0              Péssimo            Pendente  
1                Médio           

## reinserindo dados da tabela feedback 


In [42]:
def clear_table(table_name, connection):
    cursor = connection.cursor()
    query = f"DELETE FROM {table_name};"
    try:
        cursor.execute(query)
        connection.commit()
    except psycopg2.DatabaseError as e:
        print(f"Erro ao limpar dados na tabela {table_name}: {e}")
        connection.rollback()
    finally:
        cursor.close()

# Exemplo de uso para limpar a tabela Feedback
clear_table('Feedback', conn)


## Eliminar coluna classificação em 'dados_feedback'

In [44]:
dados_feedback = dados_feedback.drop(columns=['classificacao'])


In [45]:
def insert_data(df, table_name, connection):
    cursor = connection.cursor()
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    try:
        for row in df.itertuples(index=False, name=None):
            cursor.execute(query, row)
        connection.commit()
    except psycopg2.DatabaseError as e:
        print(f"Erro ao inserir dados na tabela {table_name}: {e}")
        connection.rollback()
    finally:
        cursor.close()

# Inserir dados na tabela Feedback
insert_data(dados_feedback, 'Feedback', conn)